In [6]:
# Chefkoch.de Website
import requests
import time
from bs4 import BeautifulSoup

CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s1'
CATEGORY      = '/Rezepte.html'
category_url = START_URL + CATEGORY

import random

def random_headers():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36',
    ]
    
    headers = {
        'User-Agent': random.choice(user_agents),
        # Add more headers as needed
    }
    
    return headers


def _get_html(url): 
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers=random_headers())
        except:
            print('Connection refused')
            time.sleep(10)
            continue
    return page.text

def _get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    total_pages = soup.find('div', class_='ck-pagination qa-pagination').find('a', class_='qa-pagination-pagelink-last').text
    return int(total_pages)

html_text_total_pages = _get_html(category_url)
total_pages = _get_total_pages(html_text_total_pages)
print('Total pages: ', total_pages)

Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused
Connection refused


KeyboardInterrupt: 